In [ ]:
import struct
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import glob
import os
import imageio.v2 as imageio
import scipy
from scipy.interpolate import interp1d
import shutil
import time
import matplotlib.cm as cm

Desempaquetamos los datos de la simulación numérica: 

In [ ]:
def desempaquetar_datos(snap):

    fp = open(snap, 'rb') # llamamos fp al archivo abierto en python. Lo abrimos en binario

    Npuntos = struct.unpack('>fif',fp.read(12))[1] #El [0] coge solo el primer valor devuelto

    Tres = struct.unpack('>fif', fp.read(12))[1]

    Tiempo = struct.unpack('>3f', fp.read(12))[1] # Tiempo medido 

    deshecho = struct.unpack('>f', fp.read (4))[0] # Pasamos de largo del separador

    Masas = [] # Lista de las masas de cada partícula de la galaxia satélite
    
    for i in range (Npuntos):
        masa = struct.unpack('>f', fp.read (4))[0]
        Masas.append(masa)
    
    deshecho = struct.unpack ('>2f', fp.read(8)) # Quitamos dos separadores

    coordx = [] #listas para reunir todas las coordenadas de las partículas
    coordy = []
    coordz = []

    for i in range (Npuntos):
        psx = struct.unpack('>f',fp.read(4))[0]
        coordx.append(psx)
        psy = struct.unpack('>f',fp.read(4))[0]
        coordy.append(psy)
        psz = struct.unpack('>f',fp.read(4))[0]
        coordz.append(psz)

    deshecho = struct.unpack ('>ff', fp.read(8))

    velocidadesx = [] #listas para incluir todas las velocidades de las partículas
    velocidadesy = []
    velocidadesz = []

    for i in range (Npuntos):
        velx = struct.unpack('>f',fp.read(4))[0]
        velocidadesx.append(velx)
        vely = struct.unpack('>f',fp.read(4))[0]
        velocidadesy.append(vely)
        velz = struct.unpack('>f',fp.read(4))[0]
        velocidadesz.append(velz)

    deshecho = struct.unpack ('>ff', fp.read(8))

    Tipo = [] #el tipo de cada partícula 
    for i in range (Npuntos):
        tipo = struct.unpack('>i', fp.read (4))[0]
        Tipo.append(tipo)

    deshecho = struct.unpack ('>ff', fp.read(8))

    Eps = [] # suavizado de cada partícula, para no caer en el cero absoluto de potencial
    for i in range (Npuntos):
        eps = struct.unpack('>f', fp.read (4))[0]
        Eps.append(eps)

    for i in range (Npuntos):
        if Tipo[i] == -1:
            Tipo[i] = 1
        
    deshecho = struct.unpack ('>f', fp.read(4)) #confirmamos que llegamos al final del archivo binario

    return(coordx, coordy, coordz, Masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, Tiempo)

Para cada SNAP desempaquetamos los datos y representamos la galaxia al completo en 3D. En todas las representaciones, primero la ejecutamos para el primer snap, y con los puntos de masa que se conservan, la ejecutamos en bucle para el resto.

In [ ]:
def representar_galaxia(carpeta):

    contador = 0
    for archivo in glob.glob(os.path.join(carpeta, "*")):
        if os.path.isfile(archivo): # recorremos el archivo de cada simulación, en el que se encuentran todos los SNAPs
            
            x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(archivo) #Información en cada SNAP

            fig = plt.figure() # Para plotear toda la galaxia
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(x, y, z, c='k', marker='o', s = 1) # Cada punto de masa es un punto negro 
            ax.set_xlim(-250,250)
            ax.set_ylim(-250,250)
            ax.set_zlim(-250,250)
            ax.set_xlabel(' ')
            ax.set_ylabel(' ')
            ax.set_zlabel(' ')
            
            tiempo = round(tiempo * (10**(-3)),3) # El tiempo medido en Gaños en cada paso temporal

            plt.title(r'Tiempo transcurrido= \n' + str(tiempo) + ' 1.05 \cdot Gyr')

            ruta_original = carpeta + archivo 

            directorio = 'Imágenes para gifs/' + str(os.path.basename(carpeta)) + '/galaxia completa' # Indicamos el nombre de la simulación para que la imagen de cada SNAP se guarde en una misma carpeta con todas las imágenes de una misma simulación 

            if not os.path.exists(directorio):
                os.makedirs(directorio) # Para crear automáticamente el directorio

            nuevo_archivo = os.path.basename(ruta_original) # Separa la carpeta del archivo y te devuelve únicamente el archivo
            
            ruta_nueva = os.path.join(directorio, nuevo_archivo) # Incluye el archivo (tail) que hemos separado, en el nuevo directorio 

            print(ruta_nueva)
            
            plt.savefig(ruta_nueva)
            
            plt.show()
            
            print("Procesando:", archivo)
            contador += 1
    
    directorio = 'Imágenes para gifs/' + str(os.path.basename(carpeta)) + '/galaxia completa'

    # Lista para almacenar las rutas de las imágenes
    imagenes = []

    # Obtener todas las imágenes del directorio
    for archivo in os.listdir(directorio):
        if archivo.endswith(".png"):
            imagenes.append(os.path.join(directorio, archivo))

    # Ordenar las imágenes alfabéticamente (esto es opcional, pero útil para mantener un orden)
    imagenes.sort()

    # Nombre del archivo de salida del GIF
    output_gif = 'Galaxia completa_' + str(os.path.basename(carpeta)) + '.gif'

    # Crear el GIF a partir de las imágenes
    with imageio.get_writer(output_gif, mode='I', duration=0.5) as writer:
        for imagen in imagenes:
            frame = imageio.imread(imagen)
            writer.append_data(frame)

    carpeta_destino = 'Imágenes y gifs/' + str(os.path.basename(carpeta)) # Para almacenar automáticamente el GIF

    shutil.move(output_gif, carpeta_destino)
    print("GIF creado y guardado con con éxito!")

Calculamos el CM y recortamos el radio para asegurar que en cada paso temporal la galaxia toma los puntos realmente válidos. A su vez, representamos el perfil de masa de la galaxia para comprobar que estamos cogiendo los puntos adecuadamente.

In [ ]:
def indeces_reales(list1, list2): # Hay que relacionar cada punto con una posición, para que se conserve entre un snap y otro
    idx = []
    for elemento in list2:
        index = list1.index(elemento)
        idx.append(index)
    return idx

In [ ]:
def masas_validas_reordenadas(masas_validas, masas_ordenadas, idx): # Ordena los puntos de masa para dejar al principio los ceros y no tenerlos en cuenta
    for i in range(len(masas_ordenadas)):
        masas_validas[idx[i]] = masas_ordenadas[i]
    
    return(masas_validas)

In [ ]:
def calculo_cm (Masas, Npuntos, x, y, z):
    
    masa_total = 0.0 # Sumamos las masas de todas las partículas de la galaxia
    
    for masa in Masas:
        masa_total += masa

    suma_xcm = 0.0
    suma_ycm = 0.0
    suma_zcm = 0.0

    for i in range (Npuntos):
        
        mx = Masas[i] * x[i] # El producto de masa por posicion para cada coordenada
        suma_xcm += mx

        my = Masas[i] * y[i]
        suma_ycm += my
        
        mz = Masas[i] * z[i]
        suma_zcm += mz
    
    xcm = suma_xcm / masa_total # Las coordenadas del centro de masas
    ycm = suma_ycm / masa_total
    zcm = suma_zcm / masa_total
    
    radios = []

    for i in range (Npuntos): # Distancia al centro de masas de cada partícula 
        distx = x[i] - xcm
        disty = y[i] - ycm
        distz = z[i] - zcm

        radio_ind = np.sqrt((distx**2) + (disty**2) + (distz**2)) 
        radios.append(radio_ind)

    r_max = 0
    
    for radio in radios:
        if radio > r_max:
            r_max = radio # Radio de la partícula más alejada
    
    return xcm, ycm, zcm, r_max, radios

In [ ]:
def comprobacion_primer_snap(snap):
    
    x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(snap)
    
    x_valido = x # En un primer momento tomamos todos los puntos de la galaxia para calcular el CM
    y_valido = y
    z_valido = z
    Masas_valido = masas
    
    Ptos_ppio = 0
    for i in range(Npuntos):
        if Masas_valido[i] != 0:
            Ptos_ppio += 1
            
    print('Al ppio del SNAP se cogen ' +str(Ptos_ppio)+  'puntos')
    
    tol = 10
    it = 0
    
    while tol > 10**(-10):
        
        xcm, ycm, zcm, r_max, radios = calculo_cm (Masas_valido, Npuntos, x_valido, y_valido, z_valido)
        r_max2 = 0.9 * r_max
     
        for i in range(Npuntos):
            if radios[i] >= r_max2: #Los que se quedan fuera

                x_valido[i] = 0.0
                y_valido[i] = 0.0
                z_valido[i] = 0.0
                Masas_valido[i] = 0.0
                radios[i] = 0.0

        xcm2, ycm2, zcm2, r_max2, radios2 = calculo_cm (Masas_valido, Npuntos, x_valido, y_valido, z_valido)
                
        x_de_cms = xcm2 - xcm
        y_de_cms = ycm2 - ycm 
        z_de_cms = zcm2 - zcm 

        tol = np.sqrt((x_de_cms**2) + (y_de_cms**2) + (z_de_cms**2))

        it = it+1


    distribucion_masas = []

    # Establecemos los índices de forma que el radio vaya de menor a mayor
    indices_ordenados = sorted(range(len(radios)), key=lambda i: radios[i])
    
    # Asociamos cada masa a su correspondiente radio en el nuevo orden
    masas_ordenadas = [Masas_valido[i] for i in indices_ordenados]
    radios_ordenados = [radios[i] for i in indices_ordenados]

    # indexamos los elementos de Masas_valido para recuperar el orden cuando eliminemos las masas sobrantes de la galaxia
    idx = indeces_reales(Masas_valido, masas_ordenadas) 

    Masa_total = 0.0
    for i in range(len(masas_ordenadas)):
        Masa_total += masas_ordenadas[i]
        distribucion_masas.append(Masa_total)

    r_regr = []
    d_regr = []
    m_regr = []
    initial_zeros = []

    P_virt = 0
    for i in range(Npuntos):
        if masas_ordenadas[i] != 0.0:
            r_regr.append(radios_ordenados[i])
            d_regr.append(distribucion_masas[i])
            m_regr.append(masas_ordenadas[i])
            P_virt += 1
        if masas_ordenadas[i] == 0.0:
            initial_zeros.append(masas_ordenadas[i])
   
    limit = 49
    slope_inicial, intercept_inicial, r_value_inicial, p_value, std_err = scipy.stats.linregress(r_regr[0:limit], d_regr[0:limit]) # La pendiente de la regresión lineal en los primeros puntos
    it = 0
    
    for i in range(len(r_regr[1:])):
        it += 1
        slope, intercept, r_value, p_value, std_error = scipy.stats.linregress(r_regr[i:i+limit], d_regr[i:i+limit])
        if slope > 0.4 * slope_inicial: # Cuando la pendiente se acerque a cero eliminamos la 'cola' de la galaxia
            continue
        else:
            break
            
    r_regr = r_regr[0:it+limit]
    d_regr = d_regr[0:it+limit]
    m_regr = m_regr[0:it+limit]

    if len(r_regr) != P_virt:
        ending = np.zeros(P_virt - len(r_regr))
        ending = ending.tolist()
        m_regr = m_regr + ending
        
    masas_ordenadas = initial_zeros + m_regr
    
    Masas_valido = masas_validas_reordenadas(Masas_valido, masas_ordenadas, idx) # Reordenamos los puntos de masa para devolver los ceros a la posición que les corresponde

    # print(Masas_valido, masas_ordenadas)
    plt.scatter(r_regr[0:limit], d_regr[0:limit], s=0.1, c='b')
    plt.scatter(r_regr[limit+1:], d_regr[limit+1:], s=0.1, c='k') # Representamos por separado los puntos cuya pendiente se ha tomado como referencia, y el resto
    plt.title('sim001/SNAP000')
    plt.xlabel('Radio [kpc]')
    plt.ylabel(r'Masa [$2 \cdot 10^{11} M_\odot$]')

    P_fin = 0
    for i in range(len(Masas_valido)):
        if Masas_valido[i] == 0.0:
            x_valido[i] = 0.0
            y_valido[i] = 0.0
            z_valido[i] = 0.0
            P_fin += 1
            

    return (Masas_valido, x_valido, y_valido, z_valido, tiempo) 

In [ ]:
def comprobacion_demas_snaps(snap, Masas_valido, x_valido, y_valido, z_valido): # Representamos los puntos que se han tomado en cada SNAP para toda la simulación

    x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(snap)

    # Npuntos1 ya viene dado por el input
    Ptos_ppio = 0
    for i in range(Npuntos): 
        if x_valido[i] != 0:
            
            Masas_valido[i]  = masas[i] # Cogemos los puntos que anteriormente formaban el CM, y representamos en ellos las coordenadas y masas de los puntos del snap actual
            x_valido[i] = x[i]
            y_valido[i] = y[i]
            z_valido[i] = z[i]
            Ptos_ppio += 1

    print('Al ppio del SNAP se cogen ' +str(Ptos_ppio)+  'puntos')
    tol = 10
    it = 0
    
    while tol > 10**(-10):
        
        xcm, ycm, zcm, r_max, radios = calculo_cm (Masas_valido, Npuntos, x_valido, y_valido, z_valido)
        r_max2 = 0.9 * r_max
     
        for i in range(Npuntos):
            if radios[i] >= r_max2: #Los que se quedan fuera

                x_valido[i] = 0.0
                y_valido[i] = 0.0
                z_valido[i] = 0.0
                Masas_valido[i] = 0.0
                radios[i] = 0.0

        xcm2, ycm2, zcm2, r_max2, radios2 = calculo_cm (Masas_valido, Npuntos, x_valido, y_valido, z_valido)
                
        x_de_cms = xcm2 - xcm
        y_de_cms = ycm2 - ycm 
        z_de_cms = zcm2 - zcm 

        tol = np.sqrt((x_de_cms**2) + (y_de_cms**2) + (z_de_cms**2))

        it = it+1


    distribucion_masas = []

    # Establecemos los índices de forma que el radio vaya de menor a mayor
    indices_ordenados = sorted(range(len(radios)), key=lambda i: radios[i])
    
    # Asociamos cada masa a su correspondiente radio en el nuevo orden
    masas_ordenadas = [Masas_valido[i] for i in indices_ordenados]
    radios_ordenados = [radios[i] for i in indices_ordenados]

    # indexamos los elementos de Masas_valido para recuperar el orden cuando eliminemos las masas sobrantes de la galaxia
    idx = indeces_reales(Masas_valido, masas_ordenadas) 

    Masa_total = 0.0
    for i in range(len(masas_ordenadas)):
        Masa_total += masas_ordenadas[i]
        distribucion_masas.append(Masa_total)

    r_regr = []
    d_regr = []
    m_regr = []
    initial_zeros = []

    P_virt = 0
    for i in range(Npuntos):
        if masas_ordenadas[i] != 0.0:
            r_regr.append(radios_ordenados[i])
            d_regr.append(distribucion_masas[i])
            m_regr.append(masas_ordenadas[i])
            P_virt += 1
        if masas_ordenadas[i] == 0.0:
            initial_zeros.append(masas_ordenadas[i])
   
    limit = 49
    slope_inicial, intercept_inicial, r_value_inicial, p_value, std_err = scipy.stats.linregress(r_regr[0:limit], d_regr[0:limit])
    it = 0
    
    for i in range(len(r_regr[1:])):
        it += 1
        slope, intercept, r_value, p_value, std_error = scipy.stats.linregress(r_regr[i:i+limit], d_regr[i:i+limit])
        if slope > 0.4 * slope_inicial:
            continue
        else:
            break
            
    r_regr = r_regr[0:it+limit]
    d_regr = d_regr[0:it+limit]
    m_regr = m_regr[0:it+limit]

    if len(r_regr) != P_virt:
        ending = np.zeros(P_virt - len(r_regr))
        ending = ending.tolist()
        m_regr = m_regr + ending
        
    masas_ordenadas = initial_zeros + m_regr
    
    Masas_valido = masas_validas_reordenadas(Masas_valido, masas_ordenadas, idx)

    # print(Masas_valido, masas_ordenadas)
    plt.scatter(r_regr[0:limit], d_regr[0:limit], s=0.1, c='b')
    plt.scatter(r_regr[limit+1:], d_regr[limit+1:], s=0.1, c='k')

    P_fin = 0
    for i in range(len(Masas_valido)):
        if Masas_valido[i] == 0.0:
            x_valido[i] = 0.0
            y_valido[i] = 0.0
            z_valido[i] = 0.0
            P_fin += 1
    
    P_fin = Npuntos - P_fin
    print('Al final del SNAP se cogen ' +str(P_fin)+  'puntos')
    return (Masas_valido, x_valido, y_valido, z_valido, tiempo, P_fin)

In [ ]:
def comprobacion_cm_iterado(carpeta):

    archivos = sorted(os.listdir(carpeta)) # Ordenamos los archivos de la carpeta
    contador = 0
    
    primer_snap = os.path.join(carpeta, archivos[1]) # Lee el SNAP000
    Masas1, x1, y1, z1, tiempo = comprobacion_primer_snap(primer_snap) # Nos devuelve los puntos que se han utilizado en el CM

    tiempo = round(tiempo * (10**(-3)),3) 
    
    plt.suptitle('Tiempo transcurrido = \n' + str(tiempo) + ' Gyr') # Vamos indicando el tiempo que ha transcurrido en cada SNAP

    plt.xlabel('Radio [kpc]')
    plt.ylabel('Masa []')
            
    directorio = 'Imágenes para gifs/' + str(os.path.basename(carpeta)) + '/comprobacion'

    if not os.path.exists(directorio): # Creamos el directorio en el que ir guardando las imágenes
        os.makedirs(directorio)

    nuevo_archivo = os.path.basename(primer_snap)
            
    ruta_nueva = os.path.join(directorio, nuevo_archivo)

    # Guardamos la imagen en el directorio 
    
    plt.savefig(ruta_nueva) 
            
    plt.show()
            
    print("Procesando:", primer_snap)   

    # Ahora hacemos lo mismo recorriendo todos los demás SNAPS 
    
    snaps_recorrer = glob.glob(os.path.join(carpeta, "*")) 
    
    for snap in snaps_recorrer[2:]:
        if os.path.isfile(snap):

            contador += 1
            
            # En cada bucle se toman los puntos que se han utilizado para calcular el CM en el momento anterior
            
            Masas1, x1, y1, z1, tiempo, P_finales = comprobacion_demas_snaps(snap, Masas1, x1, y1, z1) 

            tiempo = round(tiempo * (10**(-3)),3) 

            if P_finales > 150:

                plt.suptitle('Tiempo transcurrido = \n' + str(tiempo) + ' Gyr')
                plt.xlabel('Radio []')
                plt.ylabel('Masa []')
            
                ruta_original = carpeta + snap
            
                directorio = 'Imágenes para gifs/' + str(os.path.basename(carpeta)) + '/comprobacion'

                if not os.path.exists(directorio):
                    os.makedirs(directorio)

                nuevo_archivo = os.path.basename(ruta_original)
            
                ruta_nueva = os.path.join(directorio, nuevo_archivo)
            
                plt.savefig(ruta_nueva)
            
                plt.show()
            
                print("Procesando:", snap)

            else: 

                print('La galaxia desaparece una vez transcurridos ' + str(tiempo) + ' Gyr')
                break
    
    # Directorio donde se encuentran las imágenes
    directorio = 'Imágenes para gifs/' + str(os.path.basename(carpeta)) + '/comprobacion'

    # Lista para almacenar las rutas de las imágenes
    imagenes = []

    # Obtenemos todas las imágenes del directorio y las incluimos en la lista
    for archivo in os.listdir(directorio):
        if archivo.endswith(".png"):
            imagenes.append(os.path.join(directorio, archivo))

    # Ordenamos las imágenes alfabéticamente (por si acaso no están bien ordenadas)
    imagenes.sort()

    # Nombre del archivo de salida del GIF
    output_gif = 'Comprobacion_' + str(os.path.basename(carpeta)) + '.gif'

    # Creamos el GIF a partir de las imágenes, convirtiendo cada una en un frame que se va incluyendo en el gif creado por image.io
    with imageio.get_writer(output_gif, mode='I', duration=0.5) as writer: 
        for imagen in imagenes:
            frame = imageio.imread(imagen)
            writer.append_data(frame)

    carpeta_destino = 'Imágenes y gifs/' + str(os.path.basename(carpeta))

    shutil.move(output_gif, carpeta_destino)
    print("GIF creado y guardado con con éxito!")

A continuación, calculamos los puntos de masa de la galaxia en un SNAP dado, para utilizarlo en los análisis.

In [ ]:
def calculo_cm_radio(x_valido, y_valido, z_valido, Masas_valido, Npuntos): # Aplicamos el código con el que se hace la comprobación, para emplearlo en todos los análisis cuando calculamos los puntos pertenecientes a la galaxia en cada SNAP
    
    tol = 10
    it = 0

    while tol > 10**(-10):
        
        xcm, ycm, zcm, r_max, radios = calculo_cm (Masas_valido, Npuntos, x_valido, y_valido, z_valido)
        r_max2 = 0.9 * r_max
     
        for i in range(Npuntos):
            if radios[i] >= r_max2: #Los que se quedan fuera

                x_valido[i] = 0.0
                y_valido[i] = 0.0
                z_valido[i] = 0.0
                Masas_valido[i] = 0.0
                radios[i] = 0.0

        xcm2, ycm2, zcm2, r_max2, radios2 = calculo_cm (Masas_valido, Npuntos, x_valido, y_valido, z_valido)
                
        x_de_cms = xcm2 - xcm
        y_de_cms = ycm2 - ycm 
        z_de_cms = zcm2 - zcm 

        tol = np.sqrt((x_de_cms**2) + (y_de_cms**2) + (z_de_cms**2))

        it = it+1


    distribucion_masas = []

    # Establecemos los índices de forma que el radio vaya de menor a mayor
    indices_ordenados = sorted(range(len(radios)), key=lambda i: radios[i])
    
    # Asociamos cada masa a su correspondiente radio en el nuevo orden
    masas_ordenadas = [Masas_valido[i] for i in indices_ordenados]
    radios_ordenados = [radios[i] for i in indices_ordenados]

    # indexamos los elementos de Masas_valido para recuperar el orden cuando eliminemos las masas sobrantes de la galaxia
    idx = indeces_reales(Masas_valido, masas_ordenadas) 

    Masa_total = 0.0
    for i in range(len(masas_ordenadas)):
        Masa_total += masas_ordenadas[i]
        distribucion_masas.append(Masa_total)

    r_regr = []
    d_regr = []
    m_regr = []
    initial_zeros = []

    P_virt = 0
    for i in range(Npuntos):
        if masas_ordenadas[i] != 0.0:
            r_regr.append(radios_ordenados[i])
            d_regr.append(distribucion_masas[i])
            m_regr.append(masas_ordenadas[i])
            P_virt += 1
        if masas_ordenadas[i] == 0.0:
            initial_zeros.append(masas_ordenadas[i])
   
    limit = 49
    slope_inicial, intercept_inicial, r_value_inicial, p_value, std_err = scipy.stats.linregress(r_regr[0:limit], d_regr[0:limit])
    it = 0
    
    for i in range(len(r_regr[1:])):
        it += 1
        slope, intercept, r_value, p_value, std_error = scipy.stats.linregress(r_regr[i:i+limit], d_regr[i:i+limit])
        if slope > 0.4 * slope_inicial:
            continue
        else:
            break
            
    r_regr = r_regr[0:it+limit]
    d_regr = d_regr[0:it+limit]
    m_regr = m_regr[0:it+limit]

    if len(r_regr) != P_virt:
        ending = np.zeros(P_virt - len(r_regr))
        ending = ending.tolist()
        m_regr = m_regr + ending
        
    masas_ordenadas = initial_zeros + m_regr
    
    Masas_valido = masas_validas_reordenadas(Masas_valido, masas_ordenadas, idx)

    P_fin = 0
    for i in range(len(Masas_valido)):
        if Masas_valido[i] == 0.0:
            x_valido[i] = 0.0
            y_valido[i] = 0.0
            z_valido[i] = 0.0
            P_fin += 1
    
    P_fin = Npuntos - P_fin
    
    return(xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, P_fin)

Ahora, representamos los puntos de la galaxia en los 3 planos del espacio euclídeo para estudiar su evolución a lo largo de una simulación

In [ ]:
def ejes_con_cm_iterado_primer_snap(snap): # Para el primer snap en el bucle, o para representar SNAPS individuales

    x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(snap)
    
    x_valido = x # En un primer momento tomamos todos los puntos de la galaxia para calcular el CM
    y_valido = y
    z_valido = z
    Masas_valido = masas
    
    xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, P_fin = calculo_cm_radio(x_valido, y_valido, z_valido, Masas_valido, Npuntos)
    
    ax1 = plt.subplot(2,2,1)
    ax2 = plt.subplot(2,2,2)
    ax3 = plt.subplot(2,2,3)
    
    ax1.scatter(y, z, s = 0.05, c= 'k', label = 'galaxia')
    ax1.scatter(ycm, zcm, s = 1, c = 'r', label = 'CM')
    # # ax1.set_xlabel('Eje y')
    ax1.set_ylabel(r'Eje z $[kpc]$')
    ax1.set_xlim(-250,250)
    ax1.set_ylim(-250,250)
    # ax1.legend()
    
    ax2.scatter(x, z, s = 0.05, c = 'k', label = 'galaxia')
    ax2.scatter(xcm, zcm, s = 1, c = 'r', label = 'CM')
    ax2.set_xlabel(r'Eje x $[kpc]$')
    # # ax2.set_ylabel('Eje z')
    ax2.set_xlim(-250,250)
    ax2.set_ylim(-250,250)
    # ax2.legend()
    
    ax3.scatter(y, x, s = 0.05, c= 'k', label = 'galaxia')
    ax3.scatter(ycm, xcm, s = 1, c = 'r', label = 'CM')
    ax3.set_xlabel(r'Eje y $[kpc]$')
    ax3.set_ylabel(r'Eje x $[kpc]$')
    ax3.set_xlim(-250,250)
    ax3.set_ylim(-250,250)

    print(P_fin)
    
    return (Masas_valido, x_valido, y_valido, z_valido, tiempo)

In [ ]:
def ejes_con_cm_iterado_demas_snaps(snap, Masas_valido, x_valido, y_valido, z_valido): # Para iterar sobre todos los demás snaps

    x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(snap)

    for i in range(Npuntos): 
        if x_valido[i] != 0:
            
            Masas_valido[i]  = masas[i] # Cogemos los puntos que anteriormente formaban el CM, y representamos en ellos las coordenadas y masas de los puntos del snap actual
            x_valido[i] = x[i]
            y_valido[i] = y[i]
            z_valido[i] = z[i]

    tol = 10
    it = 0
    
    xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, P_fin = calculo_cm_radio(x_valido, y_valido, z_valido, Masas_valido, Npuntos)
    
    ax1 = plt.subplot(2,2,1)
    ax2 = plt.subplot(2,2,2)
    ax3 = plt.subplot(2,2,3)
    
    ax1.scatter(y, z, s = 0.05, c= 'k', label = 'galaxia')
    ax1.scatter(ycm, zcm, s = 1, c = 'r', label = 'CM')
    # # ax1.set_xlabel('Eje y')
    ax1.set_ylabel(r'Eje z $[kpc]$')
    ax1.set_xlim(-250,250)
    ax1.set_ylim(-250,250)
    # ax1.legend()
    
    ax2.scatter(x, z, s = 0.05, c = 'k', label = 'galaxia')
    ax2.scatter(xcm, zcm, s = 1, c = 'r', label = 'CM')
    ax2.set_xlabel(r'Eje x $[kpc]$')
    # # ax2.set_ylabel('Eje z')
    ax2.set_xlim(-250,250)
    ax2.set_ylim(-250,250)
    # ax2.legend()
    
    ax3.scatter(y, x, s = 0.05, c= 'k', label = 'galaxia')
    ax3.scatter(ycm, xcm, s = 1, c = 'r', label = 'CM')
    ax3.set_xlabel(r'Eje y $[kpc]$')
    ax3.set_ylabel(r'Eje x $[kpc]$')
    ax3.set_xlim(-250,250)
    ax3.set_ylim(-250,250)
    # ax3.legend()

    print(P_fin)
    
    return (Masas_valido, x_valido, y_valido, z_valido, tiempo, P_fin)

In [ ]:
def plotear_ejes(carpeta):

    inicio = time.time()
    archivos = sorted(os.listdir(carpeta)) # Ordenamos los archivos de la carpeta
    contador = 0
    
    primer_snap = os.path.join(carpeta, archivos[1]) # Lee el SNAP000
    Masas1, x1, y1, z1, tiempo = ejes_con_cm_iterado_primer_snap(primer_snap) # Nos devuelve los puntos que se han utilizado en el CM

    tiempo = round(tiempo * (10**(-3)),3) 
    
    plt.suptitle(str(os.path.basename(carpeta)) + '; Tiempo transcurrido = \n' + str(tiempo) + ' Gyr') # Vamos indicando el tiempo que ha transcurrido en cada SNAP

    directorio = 'Imágenes para gifs/' + str(os.path.basename(carpeta)) + '/Ejes'

    if not os.path.exists(directorio): # Creamos el directorio en el que ir guardando las imágenes
        os.makedirs(directorio)

    nuevo_archivo = os.path.basename(primer_snap)
            
    ruta_nueva = os.path.join(directorio, nuevo_archivo)

    # Guardamos la imagen en el directorio 
    
    plt.savefig(ruta_nueva) 
            
    plt.show()
            
    print("Procesando:", primer_snap)   

    # Ahora hacemos lo mismo recorriendo todos los demás SNAPS 
    
    snaps_recorrer = glob.glob(os.path.join(carpeta, "*")) 
    
    for snap in snaps_recorrer[2:]:
        if os.path.isfile(snap):

            contador += 1
            
            # En cada bucle se toman los puntos que se han utilizado para calcular el CM en el momento anterior
            
            Masas1, x1, y1, z1, tiempo, P_finales = ejes_con_cm_iterado_demas_snaps(snap, Masas1, x1, y1, z1) 

            tiempo = round(tiempo * (10**(-3)),3) 

            if P_finales > 150:

                plt.suptitle(str(os.path.basename(carpeta)) + '; Tiempo transcurrido = \n' + str(tiempo) + ' Gyr')
            
                ruta_original = carpeta + snap
            
                directorio = 'Imágenes para gifs/' + str(os.path.basename(carpeta)) + '/Ejes'

                if not os.path.exists(directorio):
                    os.makedirs(directorio)

                nuevo_archivo = os.path.basename(ruta_original)
            
                ruta_nueva = os.path.join(directorio, nuevo_archivo)
            
                plt.savefig(ruta_nueva)
            
                plt.show()
            
                print("Procesando:", snap)

            else: 

                print('La galaxia desaparece una vez transcurridos ' + str(tiempo) + ' Gyr')
                break
    
    # Directorio donde se encuentran las imágenes
    directorio = 'Imágenes para gifs/' + str(os.path.basename(carpeta)) + '/Ejes'

    # Lista para almacenar las rutas de las imágenes
    imagenes = []

    # Obtenemos todas las imágenes del directorio y las incluimos en la lista
    for archivo in os.listdir(directorio):
        if archivo.endswith(".png"):
            imagenes.append(os.path.join(directorio, archivo))

    # Ordenamos las imágenes alfabéticamente (por si acaso no están bien ordenadas)
    imagenes.sort()

    # Nombre del archivo de salida del GIF
    output_gif = 'Ejes_' + str(os.path.basename(carpeta)) + '.gif'

    # Creamos el GIF a partir de las imágenes, convirtiendo cada una en un frame que se va incluyendo en el gif creado por image.io
    with imageio.get_writer(output_gif, mode='I', duration=0.5) as writer: 
        for imagen in imagenes:
            frame = imageio.imread(imagen)
            writer.append_data(frame)
    
    carpeta_destino = 'Imágenes y gifs/' + str(os.path.basename(carpeta))

    shutil.move(output_gif, carpeta_destino)
    print("GIF creado y guardado con con éxito!")

    fin = time.time()
    print('La ejecución ha tardado ' + str(np.round((fin-inicio)/60, 2)) + ' minutos')

Ahora calculamos el perfil de masa de la galaxia con el radio de la órbita en cada punto.

In [ ]:
def perfil_de_masa_primer_snap(snap): 
    
    x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(snap)

    x_valido = x # En un primer momento tomamos todos los puntos de la galaxia para calcular el CM
    y_valido = y
    z_valido = z
    Masas_valido = masas
        
    xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, P_fin = calculo_cm_radio(x_valido, y_valido, z_valido, Masas_valido, Npuntos)

    # Calculamos la distancia a la galaxia central, para luego representar las órbitas
    distancia_galaxia_central = np.sqrt( xcm**2 + ycm**2 + zcm**2 )

    return (Masas_valido, x_valido, y_valido, z_valido, distancia_galaxia_central, tiempo)

In [ ]:
def perfil_de_masa_demas_snaps(snap, Masas_valido, x_valido, y_valido, z_valido): # Para iterar sobre todos los demás snaps

    x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(snap)

    for i in range(Npuntos): 
        if x_valido[i] != 0:
            
            Masas_valido[i]  = masas[i] # Cogemos los puntos que anteriormente formaban el CM, y representamos en ellos las coordenadas y masas de los puntos del snap actual
            x_valido[i] = x[i]
            y_valido[i] = y[i]
            z_valido[i] = z[i]

    xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, P_fin = calculo_cm_radio(x_valido, y_valido, z_valido, Masas_valido, Npuntos)

    distancia_galaxia_central = np.sqrt( xcm**2 + ycm**2 + zcm**2 )

    return (Masas_valido, x_valido, y_valido, z_valido, distancia_galaxia_central, tiempo, P_fin)

In [ ]:
# Las masas son aquellas que han acabado dentro del radio del CM, es decir, las que forman la lista Masas_valido en cada snap

def perfil_de_masa(carpeta):

    inicio = time.time()
    archivos = sorted(os.listdir(carpeta)) # Ordenamos los archivos de la carpeta
    contador = 0
    
    masas_tot = 0.0
    lista_masas_tot = [] 
    lista_tiempos = []
    dist_central = [] # Para almacenar las distancias a la galaxia central (o pozo de potencial)

    primer_snap = os.path.join(carpeta, archivos[1]) # Lee el SNAP000
    # Nos devuelve los puntos que se han utilizado en el CM, y la distancia al centro del pozo de potencial en ese punto
    Masas1, x1, y1, z1, dist, tiempo = perfil_de_masa_primer_snap(primer_snap) 

    dist_central.append(dist)
    
    for masa in Masas1:
        masas_tot += masa

    lista_masas_tot.append(masas_tot) # Añadimos como primer elemento la masa total en el primer SNAP

    tiempo = round(tiempo * (10**(-3)),3) 

    lista_tiempos.append(tiempo) # Añadimos a la lista de tiempos el tiempo en el que se mide

    print("Procesando:", primer_snap)   

    # Ahora hacemos lo mismo recorriendo todos los demás SNAPS

    snaps_recorrer = glob.glob(os.path.join(carpeta, "*")) 

    for snap in snaps_recorrer[2:]:
        if os.path.isfile(snap):

            contador += 1
            
            # En cada bucle se toman los puntos que se han utilizado para calcular el CM en el momento anterior
            
            Masas1, x1, y1, z1, dist, tiempo, P_finales = perfil_de_masa_demas_snaps(snap, Masas1, x1, y1, z1) 
            
            dist_central.append(dist)
            
            masas_tot = 0.0
            
            for masa in Masas1:
                masas_tot += masa

            lista_masas_tot.append(masas_tot)
                
            tiempo = round(tiempo * (10**(-3)),3) 
            
            lista_tiempos.append(tiempo) # Añadimos a la lista de tiempos el tiempo en el que se mide
            
            if P_finales > 150:
                
                print("Procesando:", snap)
                continue
                
                
            else: 
                
                ultimo_tiempo = lista_tiempos[-1]
                print('Después de ' + str(ultimo_tiempo) + ' Gyr, la galaxia ha desaparecido')
                break
    
    fig, ax1 = plt.subplots()
    fig.suptitle(os.path.basename(carpeta))
    ax2 = ax1.twinx()

    ax1.scatter(lista_tiempos, lista_masas_tot, c = 'k', s = 0.1, label = 'Masa satélite')
    ax2.plot(lista_tiempos, dist_central, c = 'b', label = 'Distancia satélite')
    fig.legend(fontsize = 10)

    ax1.set_xlabel(r'Tiempo transcurrido $[1.05 \cdot Gyr]$')
    ax1.set_ylabel(r'Masa de la galaxia $[2 \cdot 10^{11} M_{sol}]$')
    ax2.set_ylabel(r'Distancia al centro de la galaxia central $[kpc]$')

    directorio = 'Imágenes y gifs/' + str(os.path.basename(carpeta))
    
    if not os.path.exists(directorio):
        os.makedirs(directorio)

    fin = time.time()
    print('la ejecución ha tardado ' + str(np.round((fin-inicio)/60, 0)) + ' minutos')
    
    plt.savefig(directorio + '/perfil_de_masa_' + str(os.path.basename(carpeta)) + '.png')
    plt.show()

En el siguiente análisis obtenemos los momentos en los que cada galaxia pierde el 25%, 50%, 75% y 100% de la masa.

In [ ]:
def mitad_de_masa_primer_snap(snap):
    
    x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(snap)

    x_valido = x # En un primer momento tomamos todos los puntos de la galaxia para calcular el CM
    y_valido = y
    z_valido = z
    Masas_valido = masas

    # Calculamos la masa máxima de la galaxia, para luego ver como va descendiendo la proporción:

    masa_maxima = 0.0
    for masa in masas:
        masa_maxima += masa

    xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, P_fin = calculo_cm_radio(x_valido, y_valido, z_valido, Masas_valido, Npuntos)

    return (masa_maxima, Masas_valido, x_valido, y_valido, z_valido, tiempo)

In [ ]:
def mitad_de_masa_demas_snaps(snap, Masas_valido, x_valido, y_valido, z_valido):

    x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(snap)
    
    for i in range(Npuntos): 
        if x_valido[i] != 0:
            
            Masas_valido[i]  = masas[i] # Cogemos los puntos que anteriormente formaban el CM, y representamos en ellos las coordenadas y masas de los puntos del snap actual
            x_valido[i] = x[i]
            y_valido[i] = y[i]
            z_valido[i] = z[i]

    xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, P_fin = calculo_cm_radio(x_valido, y_valido, z_valido, Masas_valido, Npuntos)

    return (Masas_valido, x_valido, y_valido, z_valido, tiempo, P_fin)

In [ ]:
# Las masas son aquellas que han acabado dentro del radio del CM, es decir, las que forman la lista Masas_valido en cada snap

def mitad_de_masa(carpeta):

    inicio = time.time()

    masas_tot = 0.0
    lista_masas_tot = [] 
    lista_tiempos = []
    
    archivos = sorted(os.listdir(carpeta)) # Ordenamos los archivos de la carpeta
    contador = 0
    
    primer_snap = os.path.join(carpeta, archivos[1]) # Lee el SNAP000
    # Nos devuelve los puntos que se han utilizado en el CM, y la distancia al centro del pozo de potencial en ese punto
    masa_maxima, Masas1, x1, y1, z1, tiempo = mitad_de_masa_primer_snap(primer_snap) 

    for masa in Masas1:
        masas_tot += masa

    lista_masas_tot.append(masas_tot) # Añadimos como primer elemento la masa total en el primer SNAP
    
    tiempo = round(tiempo * (10**(-3)),3)

    lista_tiempos.append(tiempo) # Añadimos a la lista de tiempos el tiempo en el que se mide
            
    print("Procesando:", primer_snap)   

    # Ahora hacemos lo mismo recorriendo todos los demás SNAPS 
    
    snaps_recorrer = glob.glob(os.path.join(carpeta, "*")) 
    
    for snap in snaps_recorrer[2:]:
        if os.path.isfile(snap):

            contador += 1
            
            # En cada bucle se toman los puntos que se han utilizado para calcular el CM en el momento anterior
            
            Masas1, x1, y1, z1, tiempo, P_finales = mitad_de_masa_demas_snaps(snap, Masas1, x1, y1, z1) 
            
            masas_tot = 0.0
            for masa in Masas1:
                masas_tot += masa

            lista_masas_tot.append(masas_tot)
            
            tiempo = round(tiempo * (10**(-3)),3)
            
            lista_tiempos.append(tiempo)

            if P_finales > 150:
                
                print("Procesando:", snap)
                continue
                
            else: 
                ultimo_tiempo = lista_tiempos[-1]
                print('Después de ' + str(ultimo_tiempo) + ' Gyr, la galaxia ha desaparecido')
                break
    
    # proporcion de masa sobre la masa total
    array_proporcion = np.array(lista_masas_tot) / masa_maxima

    f = interp1d(array_proporcion, lista_tiempos, assume_sorted = False)


    if 0.0 < array_proporcion[-1] < 0.25:

        f = interp1d(array_proporcion, lista_tiempos, assume_sorted = False)
        
        cuarto_masa = f(0.25)
        mitad_masa = f(0.5)
        tres_cuartos = f(0.75)

        cuarto_masa = np.round(cuarto_masa,2)
        mitad_masa = np.round(mitad_masa,2)
        tres_cuartos = np.round(tres_cuartos,2)
        
        plt.scatter(lista_tiempos, array_proporcion, c = 'k', s = 0.1, label = 'Masa satélite')
        
        plt.plot(cuarto_masa, 0.25, 'x', color = 'maroon', markersize = 7, label = 'Cuarto de masa = ' + str(cuarto_masa) + ' Gyr')
        plt.plot(mitad_masa, 0.5, 'x', color = 'red', markersize = 7, label = 'Mitad de masa = ' + str(mitad_masa) + ' Gyr')
        plt.plot(tres_cuartos, 0.75, 'x', color = 'lightcoral', markersize = 7, label = 'Tres cuartos de masa = ' + str(tres_cuartos) + ' Gyr')
        
        plt.plot(lista_tiempos, array_proporcion, c = 'b')
        plt.legend(fontsize = 10)

        plt.suptitle(os.path.basename(carpeta))
        plt.xlabel(r'Tiempo transcurrido [$1.05 \cdot Gyr$]')
        plt.ylabel(r'Proporción de masa') # Añadir unidades

        directorio = 'Imágenes y gifs/' + str(os.path.basename(carpeta))
    
        if not os.path.exists(directorio):
            os.makedirs(directorio)
            
        plt.savefig(directorio + '/mitad_de_masa_' + str(os.path.basename(carpeta)) + '.png')
        plt.show()
        
    elif 0.25 < array_proporcion[-1] < 0.5:

        f = interp1d(array_proporcion, lista_tiempos, assume_sorted = False)
        
        mitad_masa = f(0.5)
        tres_cuartos = f(0.75)

        mitad_masa = np.round(mitad_masa,2)
        tres_cuartos = np.round(tres_cuartos,2)
        
        plt.scatter(lista_tiempos, array_proporcion, c = 'k', s = 0.1, label = 'Masa satélite')
        
        plt.plot(mitad_masa, 0.5, 'x', color = 'red', markersize = 7, label = 'Mitad de masa = ' + str(mitad_masa) + ' Gyr')
        plt.plot(tres_cuartos, 0.75, 'x', color = 'lightcoral', markersize = 7, label = 'Tres cuartos de masa = ' + str(tres_cuartos) + ' Gyr')
        
        plt.plot(lista_tiempos, array_proporcion, c = 'b')
        plt.legend(fontsize = 10)

        plt.suptitle(os.path.basename(carpeta))
        plt.xlabel(r'Tiempo transcurrido [$1.05 \cdot Gyr$]')
        plt.ylabel(r'Proporción de masa') 

        directorio = 'Imágenes y gifs/' + str(os.path.basename(carpeta))
    
        if not os.path.exists(directorio):
            os.makedirs(directorio)
            
        plt.savefig(directorio + '/mitad_de_masa_' + str(os.path.basename(carpeta)) + '.png')
        plt.show()

    elif 0.5 < array_proporcion[-1] < 0.75:

        f = interp1d(array_proporcion, lista_tiempos, assume_sorted = False)
        
        tres_cuartos = f(0.75)

        tres_cuartos = np.round(tres_cuartos,2)
        
        plt.scatter(lista_tiempos, array_proporcion, c = 'k', s = 0.1, label = 'Masa satélite')
        
        plt.plot(tres_cuartos, 0.75, 'x', color = 'lightcoral', markersize = 7, label = 'Tres cuartos de masa = ' + str(tres_cuartos) + ' Gyr')
        
        plt.plot(lista_tiempos, array_proporcion, c = 'b')
        plt.legend(fontsize = 10)

        plt.suptitle(os.path.basename(carpeta))
        plt.xlabel(r'Tiempo transcurrido [$1.05 \cdot Gyr$]')
        plt.ylabel(r'Proporción de masa') 

        directorio = 'Imágenes y gifs/' + str(os.path.basename(carpeta))
    
        if not os.path.exists(directorio):
            os.makedirs(directorio)
            
        plt.savefig(directorio + '/mitad_de_masa_' + str(os.path.basename(carpeta)) + '.png')
        plt.show()
        
    elif array_proporcion[-1] > 0.75: 

        plt.scatter(lista_tiempos, array_proporcion, c = 'k', s = 1, label = 'Masa satélite')
        plt.plot(lista_tiempos, array_proporcion, c = 'b')
        plt.legend(fontsize = 10)

        plt.xlabel(r'Tiempo transcurrido [$1.05 \cdot Gyr$]')
        plt.ylabel(r'Proporción de masa') 

        ultimo_tiempo = round(lista_tiempos[-1],2)
        plt.suptitle(os.path.basename(carpeta))
        print('Después de ' + str(ultimo_tiempo) + ' Gyr, la galaxia aún no ha perdido la cuarta parte de su masa') 

        directorio = 'Imágenes y gifs/' + str(os.path.basename(carpeta))
    
        if not os.path.exists(directorio):
            os.makedirs(directorio)
            
        plt.savefig(directorio + '/mitad_de_masa_' + str(os.path.basename(carpeta)) + '.png')
        plt.show()

    
    fin = time.time()
    print('la ejecución ha tardado ' + str(np.round((fin-inicio)/60, 0)) + ' minutos')
    
    return(array_proporcion[-1])

Calculamos finalmente la trayectoria del cm 

In [ ]:
def trayectoria_primer_snap(snap):
    
    x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(snap)

    x_valido = x # En un primer momento tomamos todos los puntos de la galaxia para calcular el CM
    y_valido = y
    z_valido = z
    Masas_valido = masas
        
    xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, P_fin = calculo_cm_radio(x_valido, y_valido, z_valido, Masas_valido, Npuntos)

    return (xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, tiempo)

In [ ]:
def trayectoria_demas_snaps(snap, Masas_valido, x_valido, y_valido, z_valido):

    x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(snap)

    for i in range(Npuntos): 
        if x_valido[i] != 0:
            
            Masas_valido[i]  = masas[i] # Cogemos los puntos que anteriormente formaban el CM, y representamos en ellos las coordenadas y masas de los puntos del snap actual
            x_valido[i] = x[i]
            y_valido[i] = y[i]
            z_valido[i] = z[i]

    xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, P_fin = calculo_cm_radio(x_valido, y_valido, z_valido, Masas_valido, Npuntos)

    return (xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, tiempo, P_fin)

In [ ]:
def trayectoria(carpeta):

    inicio = time.time()
    lista_xcm = []
    lista_ycm = []
    lista_zcm = []
    lista_tiempo = []
    
    archivos = sorted(os.listdir(carpeta)) # Ordenamos los archivos de la carpeta
    contador = 0
    
    primer_snap = os.path.join(carpeta, archivos[1]) # Lee el SNAP000
    xcm, ycm, zcm, Masas1, x1, y1, z1, tiempo = trayectoria_primer_snap(primer_snap) # Nos devuelve los puntos que se han utilizado en el CM

    lista_xcm.append(xcm)
    lista_ycm.append(ycm)
    lista_zcm.append(zcm)

    tiempo = round(tiempo * (10**(-3)),3)
    lista_tiempo.append(tiempo)
    
    print("Procesando:", primer_snap)
    
    snaps_recorrer = glob.glob(os.path.join(carpeta, "*"))
    
    for snap in snaps_recorrer[2:]:
        if os.path.isfile(snap):

            contador += 1
            
            # En cada bucle se toman los puntos que se han utilizado para calcular el CM en el momento anterior
            
            xcm, ycm, zcm, Masas1, x1, y1, z1, tiempo, P_finales = trayectoria_demas_snaps(snap, Masas1, x1, y1, z1) 
            
            lista_xcm.append(xcm)
            lista_ycm.append(ycm)
            lista_zcm.append(zcm)

            tiempo = round(tiempo * (10**(-3)),3)
            lista_tiempo.append(tiempo)

            if P_finales > 150:
                
                print("Procesando:", snap)
                continue
                
            else: 
                
                ultimo_tiempo = lista_tiempo[-1]
                print('Después de ' + str(ultimo_tiempo) + ' Gyr, la galaxia ha desaparecido')
                break
            

    plt.suptitle(os.path.basename(carpeta))
    
    ax1 = plt.subplot(2,2,1)
    ax2 = plt.subplot(2,2,2)
    ax3 = plt.subplot(2,2,3)

    ax1.scatter(lista_ycm, lista_zcm, s = 1, c = np.array(np.linspace(0,len(lista_xcm),len(lista_xcm))), cmap = 'coolwarm')
    # # ax1.set_xlabel('Eje y')
    ax1.set_ylabel(r'Eje z $[kpc]$')
    ax1.set_xlim(-70,70)
    ax1.set_ylim(-70,70)


    ax2.scatter(lista_xcm, lista_zcm, s = 1, c = np.array(np.linspace(0,len(lista_xcm),len(lista_xcm))), cmap = 'coolwarm')
    ax2.set_xlabel(r'Eje x $[kpc]$')
    # # ax2.set_ylabel('Eje z')
    ax2.set_xlim(-70,70)
    ax2.set_ylim(-70,70)

    ax3.scatter(lista_ycm, lista_xcm, s = 1, c = np.array(np.linspace(0,len(lista_xcm),len(lista_xcm))), cmap = 'coolwarm')
    ax3.set_xlabel(r'Eje y $[kpc]$')
    ax3.set_ylabel(r'Eje x $[kpc]$')
    ax3.set_xlim(-70,70)
    ax3.set_ylim(-70,70)
    
    norm = plt.Normalize(0, lista_tiempo[-1])  # Nuestro intervalo temporal
    sm = cm.ScalarMappable(cmap='coolwarm', norm=norm)
    sm.set_array([])  # This line is necessary, even though we don't use any actual data

   
    cbar = plt.colorbar(sm, ax=[ax1, ax2, ax3])  # Para todos los ejes

    cbar.set_label(r'Tiempo transcurrido $[1.05 \cdot Gyr]$')

    directorio = 'Imágenes y gifs/' + str(os.path.basename(carpeta))
    
    if not os.path.exists(directorio):
        os.makedirs(directorio)
            
    plt.savefig(directorio + '/trayectoria_' + str(os.path.basename(carpeta)) + '.png')
    plt.show()

    fin = time.time()
    print('La ejecución ha tardado ' + str(np.round((fin-inicio)/60, 0)) + ' minutos')

In [ ]:
En último lugar, esta función calcula la evoluci momento angular 

In [ ]:
def momento_angular(M, masas, xcm, ycm, zcm, vx, vy, vz, Npuntos): # Recoge las coordenadas y las velocidades como listas

    sumx = 0.0
    sumy = 0.0
    sumz = 0.0
    
    for i in range(Npuntos):
        
        sumx += (masas[i] * vx[i])
        sumy += (masas[i] * vy[i])
        sumz += (masas[i] * vz[i])

    vx_cm = sumx / M
    vy_cm = sumy / M
    vz_cm = sumz / M

    px = M * vx_cm
    py = M * vy_cm
    pz = M * vz_cm


    list_r = xcm, ycm, zcm
    vec_r = np.array(list_r)
    list_p = px,py,pz
    vec_p = np.array(list_p)

    L = np.cross(vec_r, vec_p)
    return (L)

In [ ]:
def momento_angular_primer_snap (snap):

    x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(snap)

    x_valido = x # En un primer momento tomamos todos los puntos de la galaxia para calcular el CM
    y_valido = y
    z_valido = z

    vx_valido = velocidadesx # En un primer momento tomamos todos los puntos de la galaxia para calcular el CM
    vy_valido = velocidadesy
    vz_valido = velocidadesz
    
    Masas_valido = masas

    xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, P_fin = calculo_cm_radio(x_valido, y_valido, z_valido, Masas_valido, Npuntos)
    
    for i in range(Npuntos): 
        if x_valido[i] == 0:

            vx_valido[i] = 0.0
            vy_valido[i] = 0.0
            vz_valido[i] = 0.0
    
    masa_total = 0
    for masa in Masas_valido:
        masa_total += masa
        
    mom_ang = momento_angular (masa_total, Masas_valido, xcm, ycm, zcm, vx_valido, vy_valido, vz_valido, Npuntos)
    
    return (Masas_valido, x_valido, y_valido, z_valido, mom_ang, tiempo)

In [ ]:
def momento_angular_demas_snaps (snap, Masas_valido, x_valido, y_valido, z_valido) :
    
    x, y, z, masas, velocidadesx, velocidadesy, velocidadesz, Npuntos, tiempo = desempaquetar_datos(snap)

    vx_valido = np.zeros(Npuntos)
    vy_valido = np.zeros(Npuntos)
    vz_valido = np.zeros(Npuntos)
    
    for i in range(Npuntos): 
        if x_valido[i] != 0:
            
            Masas_valido[i]  = masas[i] # Cogemos los puntos que anteriormente formaban el CM, y representamos en ellos las coordenadas y masas de los puntos del snap actual
            x_valido[i] = x[i]
            y_valido[i] = y[i]
            z_valido[i] = z[i]

            vx_valido[i] = velocidadesx[i]
            vy_valido[i] = velocidadesy[i]
            vz_valido[i] = velocidadesz[i]

    xcm, ycm, zcm, Masas_valido, x_valido, y_valido, z_valido, P_fin = calculo_cm_radio(x_valido, y_valido, z_valido, Masas_valido, Npuntos)

    masa_total = 0
    for masa in Masas_valido:
        masa_total += masa
        
    for i in range(Npuntos): 
        if x_valido[i] == 0:

            vx_valido[i] = 0.0
            vy_valido[i] = 0.0
            vz_valido[i] = 0.0

    mom_ang = momento_angular (masa_total, Masas_valido, xcm, ycm, zcm, vx_valido, vy_valido, vz_valido, Npuntos)
            
    return (Masas_valido, x_valido, y_valido, z_valido, mom_ang, tiempo, P_fin)

In [ ]:
def momento_angular_scatter (carpeta):

    inicio = time.time()
    
    lista_ang= [] # Creamos una lista a la que añadir el módulo del momento angular
    lista_tiempo = []
    
    archivos = sorted(os.listdir(carpeta)) # Ordenamos los archivos de la carpeta
    contador = 0
    
    primer_snap = os.path.join(carpeta, archivos[1]) # Lee el SNAP000
    
    Masas1, x1, y1, z1, mom_ang, tiempo = momento_angular_primer_snap(primer_snap) # Nos devuelve los puntos que se han utilizado en el CM

    modulo_L = np.sqrt((mom_ang[0] ** 2) + (mom_ang[1] ** 2) + (mom_ang[2] ** 2 ))
    Lz = mom_ang[2]
    angulo = abs(Lz / modulo_L)

    lista_ang.append(angulo)

    tiempo = round(tiempo * (10**(-3)),3)
    lista_tiempo.append(tiempo)
    
    print("Procesando:", primer_snap)
    
    snaps_recorrer = glob.glob(os.path.join(carpeta, "*"))

    for snap in snaps_recorrer[2:]:
        if os.path.isfile(snap):

            contador += 1
            
            # En cada bucle se toman los puntos que se han utilizado para calcular el CM en el momento anterior
            
            Masas1, x1, y1, z1, mom_ang, tiempo, P_finales = momento_angular_demas_snaps(snap, Masas1, x1, y1, z1) 
            
            modulo_L = np.sqrt((mom_ang[0] ** 2) + (mom_ang[1] ** 2) + (mom_ang[2] ** 2 ))
            Lz = mom_ang[2]
            angulo = abs(Lz / modulo_L)

            lista_ang.append(angulo)

            tiempo = round(tiempo * (10**(-3)),3)
            lista_tiempo.append(tiempo)

            if P_finales > 150:
                
                print("Procesando:", snap)
                continue
                
            else: 
                
                ultimo_tiempo = lista_tiempo[-1]
                print('Después de ' + str(ultimo_tiempo) + ' Gyr, la galaxia ha desaparecido')
                break

    
    plt.plot(lista_tiempo, lista_ang, 'k')

    plt.suptitle(os.path.basename(carpeta))

    plt.xlabel(r'Tiempo transcurrido [$1.05 \cdot Gyr$]')
    plt.ylabel(r'Coseno del ángulo formado entre el momento angular el eje z') # Añadir unidades

    plt.ylim(-0.1,1.1)
    directorio = 'Imágenes y gifs/' + str(os.path.basename(carpeta))
    
    if not os.path.exists(directorio):
        os.makedirs(directorio)
            
    plt.savefig(directorio + '/momento_angular_' + str(os.path.basename(carpeta)) + '.png')
    plt.show()
    
    fin = time.time()
    print('La ejecución ha tardado ' + str(np.round((fin-inicio)/60, 0)) + ' minutos')

In [ ]:
Presentamos a continuación un ejemplo de cómo se han utilizado los resultados de la función mitad_de_masa para comparar la pérdida de masa en distintas velocidades.

In [ ]:
x_041 = [0.0, 11.55, 15.0] # Entre 50 y 75
y_041 = [1.0, 0.75, 0.69695]

x_042 = [0.0, 2.6, 4.88, 8.56, 13.42] # Se queda en 0
y_042 = [1.0, 0.75, 0.5, 0.25, 0.0]

x_043 = [0.0, 15.0] # Más de 75
y_043 = [1.0, 0.8343]

x_044 = [0.0, 1.47, 2.63, 3.98, 5.86] # Se queda en 0
y_044 = [1.0, 0.75, 0.5, 0.25, 0.0]

plt.plot(x_041, y_041, 'x', color = 'hotpink')
plt.plot(x_041, y_041, color = 'hotpink', label = 'sim041')
plt.plot(x_042, y_042, 'x', color = 'deeppink')
plt.plot(x_042, y_042, color = 'deeppink', label = 'sim042')
plt.plot(x_043, y_043, 'x', color = 'pink')
plt.plot(x_043, y_043, color = 'pink', label = 'sim043')
plt.plot(x_044, y_044, 'x', color = 'mediumvioletred')
plt.plot(x_044, y_044, color = 'mediumvioletred', label = 'sim044')


plt.yticks(np.arange(0, 1.25, step=0.25))
plt.ylim(-0.1,1.1)
plt.xlim(0,15)
plt.xlabel(r'Tiempo $[1.05 \cdot Gyr]$')
plt.ylabel('Proporción de masa')
plt.legend()
plt.title('sim 04*')

plt.savefig('Imágenes y gifs/Artículo/Masa velocidades')

plt.show()